# PhyliCS: tutorial

PhyliCS is a pipeline for **multi-sample** copy-number variation (CNV) analysis on
**single-cell DNA** sequencing data. It allows to quantify **intra-tumor heterogeneity** and to investigate **temporal and spatial evolution** of tumors.

Here, we will describe the main components of the pipeline and show how to use them to re-produce the use-cases presented in the Supplementary materials of our paper. Specifically, we will execute the second use-case *(Temporal evolution)* while we will, just, list the commands needed to reproduce the first one *(Spatial intra-tumor heterogeneity)*. 

## The pipeline

PhyliCS is a command line program which can be executed by typing `phylics` followed by some arguments. In the following, we will describe the meaning of such arguments. 

**Execution modes (required)**

One of the following execution modes MUST be specified.

| Argument        | Description     |      
| :------------- | :------------- | 
| `--run`      | Runs the three analysis steps (CNV calling, single-sample, multi-sample). | 
| `--run_10x_preproc` | Runs 10x data pre-processing module. Only single-sample execution is available for this option.|    
| `--run_cell_filtering` | Runs the cell filtering module. Only single-sample execution is available for this option.  Note that at least one of the two options, "--intervals" and "--values", must contain values to make this command effective.|

**Single-stage execution options**

The following options may be specified to run only one of the pipeline analysis stages when executing the --run mode.

| Argument | Description |
| :------| :-----------|
| `--run_cnv` | (*Optional* with `--run`) Runs only the CNV analysis. &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|
| `--run_single` | (*Optional* with `--run`) Runs only the single-sample analysis. |
| `--run_multiple` | (*Optional* with `--run`) Runs only the multiple-sample analysis. |

**Other execution options**

| Argument | Description |
| :----- | :---------- | 
|`--input_dirs` | (*Required*)  List of couple of strings separated by ":": each item of the  list corresponds to one input sample and specifies its name and the path to directory containing Ginkgo output files (`SegCopy`, `results.txt`). Note that sample name and input directory path cannot contain ":". |
|  `--method` | (*Optional*) Hierarchical clustering method (default = complete). |
|  `--metric`  | (*Optional*) Distance metric for clustering. |
|  `--output_path` | (*Required* if not `--run_cnv`) Path to the location where the output directories for the different analysis stages must be created. When running `--run_cnv` it is not necessary to specifiy the output path, since Ginkgo stores the output files in the the input directory itself. |
|  `--output_prefix` | (*Optional*) A string to be pre-pended to the names automatically generated for the output directories.|
|  `--tasks` | (*Optional*) Maximum number of tasks to be run in parallel. It allows to execute single-sample analyses in parallel and to parallelize the permutation test execution for the heterogeneity score computation. | 
|  `--seed` | (*Optional*) Seed to initialize the pseudo-random generator used to perform the permutation test. |
|  `--n_permutations` | (*Optional*) Number of permutations to execute the permutation test for the heterogeneity score computation. |
|  `--reclust` | (*Optional*) If this option is specified, only the clustering part is executed with the specified number of clusters (see --n_clusters option), unless --reinit option is specified. |
|  `--reinit`  | (*Optional*) This option has effect only if combined with the --reclust option. It allows to recompute the entire analysis and then recluster with the specified number of clusters.|
|  `--intervals`  | (*Required* with `--run_cell_filtering` if ` --values` not specified) List of of mean ploidy intervals. Cells which mean ploidies are in the specified ranges are filtered out
|  `--values`  | (*Required* with `--run_cell_filtering` if ` --intervals` not specified) List of of mean ploidy values. Cells which mean ploidies are equal to the specified ones are filtered out. |
|  `--genome`  | (*Required* with `--run` and `--run_cnvs`) Chosen genome. | 
|  `--binning` | (*Required* with `--run` and `--run_cnvs`) A complex value made of the concatenation of <ul><li> type: variable or fixed (bins. Variable refers to amount of mappable genome, recommended); </li><li> size: bin size; </li><li> read-length: mapped reads length; </li><li> aligner: bowtie or bwa. </li></ul> The read-length and aligner refer to the simulations of re-mapping reads of that length with that aligner on the whole genome. This is used to calculate bins of "mappable" (i.e. variable) genomDescriptione.The resulting value is the name of a file under ginkgo/genomes/$choosen_genome/original/ with the bin coordinates.|

**Optional arguments**

| Argument | Description |
| :----- | :---------- | 
|`--verbose` | (*Optional*) Verbose execution. &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;| 
|`-h`, `--help` | (*Optional*) Show the help message and exit. | 

## Use-case: Temporal evolution
Here, we show how to use PhyliCS to investigate temporal evolution of CNVs. To this purpose, we take advantage of the results of one of the CNV analyses performed by Garvin et. al [1] to validate their tool (*Ginkgo*). Specifically, they analyzed the single-cell data of two samples coming from a breast tumor and its liver metastasis (T16P/M), used by Navin et al. [2] for their study on intra-tumor heterogeneity characterization. Since CNV calls are, publicly, available in Ginkgo github repository, we will skip the CNV calling stage and we will move directly to the analysis.

### Single-sample analysis
This stage runs on Ginkgo results. Specifically, it takes as input files,

- `SegCopy`: a table containing he copy-number computed for each cell and position in the genome.
- `results.txt.`: a table containing some useful statistics about each cell, such as its mean ploidy. 

Now, in order to perform single-sample analysis, type:

In [1]:
!mkdir data/navin_out 
!phylics --run --run_single --input_dirs primary:data/navin_primary metastasis:data/navin_metastasis --output_path data/navin_out --verbose

Successfully created the directory data/navin_out/primary_post_CNV 
Successfully created the directory data/navin_out/metastasis_post_CNV 
[single_sample_post_analysis]  Complete analysis
--------------------------------------------------------------------------------
[single_sample_post_analysis]  Computing heatmap and phylogenetic tree (method = complete, metric = euclidean)
[single_sample_post_analysis] -- cophenet coefficient: 0.9922210550775276
--------------------------------------------------------------------------------
[single_sample_post_analysis]  Plotting mean ploidy distribution
--------------------------------------------------------------------------------
[single_sample_post_analysis]  Computing mean CNV profile
[single_sample_post_analysis] -- mean ploidy = 2.884057
--------------------------------------------------------------------------------
[single_sample_post_analysis]  Computing the optimal number of clusters
[single_sample_post_analysis] -- n_clusters = 2 - Th

**Output**

This command will generate, at the specified output path, one folder for each of the analyzed samples where all the output files will be stored. Specifically, 

| Directory | Content| 
| :---------| :------|
|`data/navin_out/<sample_name>_post_CNV`|  <ul><li><code>heatmap.png</code>: heatmap and dendrogram computed by the phylogenetic algorithm. </li><li><code>mean_cnv.png</code>: average copy-number plot. It shows which is the average copy-number, computed on all cells, for each genome position.</li><li><code>mean_ploidy_distribution.svg</code>: mean ploidy density distribution plot. The mean ploidy is the mean copy-number of each single cell and this plot shows how the mean ploidies are distributed. It allows to high-light groups of pseudo-diploids cells.</li><li> <code>silhouette_results.png</code>: silhouette plot for each of the tested K's.</li><li> <code>per_k_silhouette_scores.csv</code>: average silhoutte scores for each of the  tested K's.</li><li> <code>silhouette_summary.png</code>: dot plot of the silhouette score for the tested K's.</li><li> <code>clusters.tsv</code>: composition and mean copy-number of the clusters built with the Silhouette method.</li><li> <code>clusters_heatmap.png</code>: heatmaps of the  clusters built with the Silhouette method.</li><li> <code>mean_cnv_clusters.png</code>: average copy-number of the genome of the  clusters built with the Silhouette method.</li></ul>

### Cell filtering
As shown in the dedicated section of the Supplementary materials, both samples contain a group of pseudo-diploid cells. To filter them out, let us take a look to the mean ploidy density plot in `mean_ploidy_distribution.svg` of the primary sample:

![Primary mean ploidy density plot](data/navin_out/primary_post_CNV/mean_ploidy_distribution.png)

We may assume that the pseudo-diploid cells have a ploidy ranging in the interval 1.5-2.3, so we filter them out. To do so, just type:
    

In [2]:
!phylics --run_cell_filtering --input_dirs primary:data/navin_primary --intervals 1.5-2.3 --output_path data/navin_out --verbose

Cell filtering execution
Successfully created the directory data/navin_out/primary_filtered 
[valid_cells]  Initial cells: 52
[valid_cells]  Filtered out cells: 33
[valid_cells]  Remaining cells: 19


Similarly, for the metastasis sample, type:

In [3]:
!phylics --run_cell_filtering --input_dirs metastasis:data/navin_metastasis --intervals 1.5-2.3 --output_path data/navin_out --verbose

Cell filtering execution
Successfully created the directory data/navin_out/metastasis_filtered 
[valid_cells]  Initial cells: 48
[valid_cells]  Filtered out cells: 25
[valid_cells]  Remaining cells: 23


**Output**
The output of this command will go in another directory, created at the specified output path. Specifically,

| Directory | Content| 
| :---------| :------|
|`data/navin_out/<sample_name>_filtered`| <ul><li><code>SegCopy</code>: filtered CNV file.</li><li><code>results.txt</code>: filtered statistics file.</li></ul>

### Multiple-sample analysis
Finally, perform multiple-sample analysis on the filtered data, type:

In [4]:
!phylics --run --run_multiple --input_dirs primary:data/navin_out/primary_filtered metastasis:data/navin_out/metastasis_filtered  --output_path data/navin_out --verbose

Successfully created the directory data/navin_out/primary_metastasis_postCNV 
[multi_sample_post_analysis]  CNV calls merging
--------------------------------------------------------------------------------
[multi_sample_post_analysis]  Complete analysis
--------------------------------------------------------------------------------
[multi_sample_post_analysis]  Heterogeneity score computation
[multi_sample_post_analysis] -- Permutation test (n_permutations = 1000)
[multi_sample_post_analysis] ---- iteration: 0
[multi_sample_post_analysis] ---- iteration: 100
[multi_sample_post_analysis] ---- iteration: 200
[multi_sample_post_analysis] ---- iteration: 300
[multi_sample_post_analysis] ---- iteration: 400
[multi_sample_post_analysis] ---- iteration: 500
[multi_sample_post_analysis] ---- iteration: 600
[multi_sample_post_analysis] ---- iteration: 700
[multi_sample_post_analysis] ---- iteration: 800
[multi_sample_post_analysis] ---- iteration: 900
[multi_sample_post_analysis] ---- Permuta

**Output** 
Also this command will generate a new directory at the specified output path where the results will be stored. Specifically,

| Directory | Content| 
| :---------| :------|
|`data/navin_out/<sample_name>_<sample_name>_post_CNV`| <ul><li><code>multi_sample_heatmap.png</code>: heatmap and dendrogram computed by the phylogenetic algorithm.</li><li><code>silhouette_results.png</code>: silhouette plot for each of the tested K's. </li><li> `per_k_silhouette_scores.csv`: average silhoutte scores for each of the  tested K's.</li><li> <code>silhouette_summary.png</code>: dot plot of the silhouette score for the tested K's.</li><li> <code>clusters.tsv</code>: composition and mean copy-number of the clusters built with the Silhouette method. </li><li> <code>clusters_heatmap.png</code>: heatmaps of the  clusters built with the Silhouette method. </li><li> <code>mean_cnv_clusters.png</code>: average copy-number of the genome of the  clusters built with the Silhouette method.</li><li> <code>het_score_violin_plot.png</code>: heterogeneity score permutation test distribution plot. </li><li><code>SegCopy_merged</code>: table containing the copy-numbers of the cells from all samples.</li></ul>
    

## Use-case: Spatial intra-tumor heterogeneity
Here, we will just show the commands you need to execute to reproduce the first use-case. We used PhyliCS to study spatial intra-tumor heterogeneity by taking in account multiple samples taken from different regions of the same tumor. We used three public datasets provided by 10x Genomics on their official website [3].

### Download data
First, you need to download the allignment data and the file containing the cell barcodes, produced by Cell Ranger DNA, for each sample. To this purpose, type:

```
wget -O data/10x_breastA/possorted_bam.bam http://s3-us-west-2.amazonaws.com/10x.files/samples/cell-dna/1.1.0/breast_tissue_A_2k/breast_tissue_A_2k_possorted_bam.bam

wget -O data/10x_breastA/per_cell_summary_metrics.csv http://cf.10xgenomics.com/samples/cell-dna/1.1.0/breast_tissue_A_2k/breast_tissue_A_2k_per_cell_summary_metrics.csv

wget -O data/10x_breastB/possorted_bam.bam http://s3-us-west-2.amazonaws.com/10x.files/samples/cell-dna/1.1.0/breast_tissue_B_2k/breast_tissue_B_2k_possorted_bam.bam

wget -O data/10x_breastB/per_cell_summary_metrics.csv http://cf.10xgenomics.com/samples/cell-dna/1.1.0/breast_tissue_B_2k/breast_tissue_B_2k_per_cell_summary_metrics.csv

wget -O data/10x_breastC/possorted_bam.bam http://s3-us-west-2.amazonaws.com/10x.files/samples/cell-dna/1.1.0/breast_tissue_C_2k/breast_tissue_C_2k_possorted_bam.bam

wget -O data/10x_breastB/per_cell_summary_metrics.csv http://cf.10xgenomics.com/samples/cell-dna/1.1.0/breast_tissue_C_2k/breast_tissue_C_2k_per_cell_summary_metrics.csv
```

### Data preparation
Then, you need to run the data-preparation module on each sample. It will call `sctools_demultiplex` to split 10x allignment files into single-cell allignemnt files, filter-out poor quality reads and multi-mappers and produce single-cell .bed files. You can do it by, simply, typing:

```
phylics --run_10x_preproc --input_dirs breast_A:data/10x_breastA --output_path data --verbose

phylics --run_10x_preproc --input_dirs breast_B:data/10x_breastB --output_path data --verbose

phylics --run_10x_preproc --input_dirs breast_C:data/10x_breastC --output_path data --verbose
```

### Data analysis
Now, you can run the complete pipeline, without going step by step. To do so, type:

```
mkdir 10x_breast_out
phylics --run --input_dirs breast_A:data/breast_A_sc breast_B:data/breast_B_sc breast_C:data/breast_C_sc --genome GrCh38 --binning variable_500000_101_bwa --output_path 10x_breast_out --tasks 8 --verbose
```

In case you prefer executing each stage separately, you may use and, properly, modify the instructions showed in the previous example, after calling the CNV stage in this way:

```phylics --run --run_cnvs --input_dirs breast_A:data/10x_breast_out/breast_A_sc breast_B:data/breast_B_sc breast_C:data/breast_C_sc --genome GrCh38 --binning variable_500000_101_bwa --verbose```



## References
1. Tyler Garvin, Robert Aboukhalil, Jude Kendall, Timour Baslan, Gurinder S Atwal, James Hicks, Michael Wigler, and Michael C Schatz. Interactive analysis and assessment of single-cell copy-number variations. Nature methods, 12(11):1058, 2015.

2. Nicholas Navin, Alexander Krasnitz, Linda Rodgers, Kerry Cook, Jennifer Meth, Jude Kendall, Michael Riggs, Yvonne Eberling, Jennifer Troge, Vladimir Grubor, et al. Inferring tumor progression from genomic heterogeneity. Genome research, 20(1):68–80, 2010.

3. 10x Genomics. 10x Genomics: Biology at True Resolution, 2019. URL https://www.10xgenomics.com.
